# File Writer - Global Fireball Exchange (GFE) Standard

This notebook writes a sample single-station meteor observation in the standard GFE format. It will prompt for an output file which then be written in GFE format with the file extension ".ecsv".

This script was originated by and is maintained by Jim Rowe of the UK Fireball Alliance, www.ukfall.org.uk 


In [7]:
# system packages
import os
import pprint
import sys

# date handling
#from datetime import datetime
from datetime import timedelta

# numerical packages
import numpy as np
#import pandas as pd
import astropy.units as u
from astropy.table import Table
#from astropy.table import Column
from astropy.time import Time
#from astropy.io import ascii

#File opening controls
from tkinter import filedialog 

# definitions of constants:
ISO_FORMAT = "%Y-%m-%dT%H:%M:%S.%f"  #defines a consistent iso date format

## Main program

In [ ]:
# Construct a data file in Standard fomat, in this case with with arbitrary contents
    
# First, construct a dictionary of metadata
print('Constructing metedata dictionary')
meta_dic = {'obs_latitude': 51.637402, # Decimal signed latitude (-90 S to +90 N)
   'obs_longitude': -0.1692,           # Decimal signed longitude (-180 W to +180 E)
   'obs_elevation': 86.0,              # Altitude in metres above MSL. Note not WGS84
   'origin': 'Nameofsoftware ver 4',   # The software which produced the data file
   'location': 'EastBarnet',           # The name of the city or location, which may have multiple cameras
   'telescope': 'NW',                  # The identifier of the particular camera within the observatory – e.g. NW or 2
   'camera_id': 'EU88877',             # The code name of the camera, likely to be network-specific
   'observer': 'H.H. Nininger',        # A person associated with the camera
   'comment': 'Just sample data',      # Any text that could be useful
   'instrument': 'Basler XY12345',     # The commercial camera model forming the basis of the camera system
   'lens': 'Nikon 4mm',                # The brand name and description of the camera lens
   'cx' : 1920,                        # Horizontal camera resolution in pixels
   'cy' : 1080,                        # Vertical camera resolution in pixels
   'photometric_band' : 'Unknown',     # The photometric band of the star catalogue
   'image_file' : 'Capture.mp4',       # The name of the original image or video
   'isodate_start_obs': '2021-04-07T03:56:40.250', # The date and time of the start of the video or exposure  
   'isodate_calib': '2021-04-07T03:56:42.590',     # The date and time corresponding to the astrometric calibration 
   'exposure_time': 5.9,               # The length of the video in seconds or the image exposure time in seconds
   'astrometry_number_stars' : 33,     # The number of stars identified and used in the astrometric calibration
   'mag_label': 'mag',                 # The label of the Magnitude column in the Point Observation data
   'no_frags': 1,                      # The number of meteoroid fragments described in this data
   'obs_az': 298.953644,               # The azimuth of the centre of the field of view in decimal degrees. North = 0, increasing to the East
   'obs_ev': 31.447386,                # The elevation of the centre of the field of view in decimal degrees. Horizon =0, Zenith = 90
   'obs_rot': -2.832725,               # Rotation of the field of view from horizontal, decimal degrees. Clockwise is positive
   'fov_horiz': 64.745132,             # Horizontal extent of the field of view, decimal degrees
   'fov_vert': 51.796231,              # Vertical extent of the field of view, decimal degrees
   }


# Create an Astropy table and add the metadata to the table
print('Creating Astropy table')
ttt = Table()        
ttt.meta.update(meta_dic)   


# Now construct the Point Observation data.  Let's assume ten points were observed

# Some imaginary initial data
obstime = '2021-04-07T03:56:41.510'   # The date and time corresponding to the point observation 
ra = 166.8385898                       # The J2000 Right Ascension of the point observation, in decimal degrees  
dec = 21.8464019                       # The J2000 Declination of the point observation, in decimal degrees  
az = 282.2609209                       # The calculated true azimuth of the point in epoch of date, expressed in decimal degrees. North = 0, increasing to the East
altitude = 18.2187545                  # The calculated true elevation of the point in epoch of date, expressed in view in decimal degrees. Horizon =0, Zenith = 90
mag = 0.6                              # The light curve – either the astronomical magnitude of the point or any other related measure, as identified by the metadata item called “mag_label”
x_image = 101.3                        # The horizontal position of the point in the image (measured from the left), measured in pixels
y_image = 587.3                        # The vertical position of the point in the image (measured from the top), measured in pixels

# Set up some empty arrays
datetime_array  = []
ra_array  = []
dec_array = []
az_array = []
altitude_array = []
mag_array = []
x_image_array = []
y_image_array = []



# Now append some data to the arrays
timestamp = Time(obstime)
no_lines = 10

for i in range(no_lines):
    print('    Creating Point Observation Data, line ', i+1)
    datetime_array.append(obstime)
    ra_array.append(ra)
    dec_array.append(dec)        
    az_array.append(az)
    altitude_array.append(altitude)
    mag_array.append(mag)
    x_image_array.append(x_image)
    y_image_array.append(y_image)
        
    # Update the invented data for the next row    
    obstime = str(timestamp + timedelta(seconds=(i+1)*0.04))  
    ra += -0.0803093222
    dec += -0.0967334666
    az += -0.006471644
    altitude += -0.1221034222
    mag = -2.3267 + 0.5107543 * abs(i-5)
    x_image += 3.5614
    x_image += 6.7867
 

# create columns in the Astropy table
print('Adding columns to table')
ttt['datetime'] = datetime_array
ttt['ra']  = ra_array  * u.degree
ttt['dec'] = dec_array * u.degree 
ttt['azimuth'] = az_array
ttt['altitude'] = altitude_array
ttt[ttt.meta['mag_label']] = mag_array
ttt['x_image'] = x_image_array
ttt['y_image'] = y_image_array


# Construct the output file name: note should include millisecs to distinguish
# between different events in the same second
output_file = obstime.replace(":", "_").replace('.','_')
output_file += '_' + 'STD' + '_' 
output_file += ttt.meta['camera_id'].replace(" ", "_")[0:15]
output_file += '.ecsv' 



print('About to prompt for output file name, default name = ',output_file)
# Now write the data to a Standard data file
out_name = filedialog.asksaveasfilename(initialfile=output_file,title = "Save file")
if out_name :
    ttt.write(out_name,overwrite=True, format='ascii.ecsv', delimiter=',')

print("Data file written: ", out_name)
# End of code            